In [3]:
import pandas as pd
import time

from selenium import webdriver # obtain the ChromeDriver compatible with the version of the browser being used
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

import chromedriver_binary  # Adds chromedriver binary to path

### Get Scraped Data

In [8]:
%pip install chromedriver-binary==118.0.5993.70

  Created wheel for chromedriver-binary: filename=chromedriver_binary-118.0.5993.70.0-py3-none-any.whl size=8580913 sha256=c73529a2dd631d6c793866fb278db99057a22feef09a0ef1f32f83d0e36a70de
  Stored in directory: /Users/caraloo/Library/Caches/pip/wheels/fd/7f/3f/309dd48b6941f29f20692d46d5cf148415a344f46f279f7d2f
Successfully built chromedriver-binary
  Attempting uninstall: chromedriver-binary
    Found existing installation: chromedriver-binary 119.0.6022.0.0
    Uninstalling chromedriver-binary-119.0.6022.0.0:
      Successfully uninstalled chromedriver-binary-119.0.6022.0.0
Note: you may need to restart the kernel to use updated packages.


In [17]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
url = "https://www.winelistasia.com/online-shop.html"
driver.maximize_window()
driver.get(url)
time.sleep(3)

i = 1
# df = pd.DataFrame(columns=['Product_Name', 'Scraped_Data_Price', 'Product_In_Stock'])
product_names = []
while i <= 15: # 15 pages on the website
    print("page: " + str(i))
    try:
        wines = driver.find_elements(By.XPATH, "//div[@class='product-grid__info']")
        for card in wines:
            wine = card.text.split('\n')
            if len(wine) >= 3: # only take the products, those with len()<3 are other parts of the page
                product_names.append(wine[0])
                """ temp = [wine[0]]
                price = wine[1].replace('SGD ','').replace(',', '').split(' ')
                if len(price) > 1: # len(list)>1 if there's discount
                    temp.append(float(price[1]))
                else:
                    temp.append(float(price[0]))
                if wine[2] == 'Sold out':
                    temp.append(False)
                else:
                    temp.append(True)
                df.loc[len(df)] = temp """
    except NoSuchElementException:
        pass
    try: # toggle pages
        button = driver.find_element(By.XPATH, "//a[@class='product-grid__paging--link product-grid--next js-products-next']")
        driver.execute_script("arguments[0].click();", button)
        time.sleep(10)
    except NoSuchElementException:
        print("The element does not exist / has not loaded yet.")
    i += 1

#df.drop_duplicates(subset="Product_Name", keep='first', inplace=True)
#df['Supplier_Name'] = 'Winelistasia'

page: 1
page: 2
page: 3
page: 4
page: 5
page: 6
page: 7
page: 8
page: 9
page: 10
page: 11
page: 12
page: 13
page: 14
page: 15


In [19]:
print(len(product_names))
# product_names

342


### Get BBW Product Data

In [41]:
bbw_products = pd.read_csv('../datasets/products.csv')
bbw_products = bbw_products[['product_name','price']]
bbw_products.set_index('product_name',inplace=True)
bbw = bbw_products.to_dict()['price']

In [50]:
print(len(bbw))
list(bbw.keys())[0:5]

2355


['Ken Forrester T Noble Late Harvest 2018 375ML',
 'Imako Constellation Chardonnay 2019',
 'Champagne Le Brun de Neuville Autolyse Noirs & Blancs Champagne Brut NV',
 'Champagne Le Brun de Neuville Millesime Extra Brut 2009',
 'Champagne Le Brun de Neuville Cote Blanche Blanc de Blancs Brut NV 375ML']

# Match Scraped Products to BBW Products

In [45]:
import re # use regex to match our keyword
from fuzzywuzzy import fuzz

In [91]:
test = pd.DataFrame(columns=['winelistasia_pdt_name', 'match_wo_regex', 'max_score_wo_regex'])

for name in product_names:
    max_score = 0
    matched_name = ''
    for wine in list(bbw.keys()):
        score = fuzz.ratio(name.lower(), wine.lower())
        if score > max_score:
            matched_name = wine
            max_score = score
    if max_score >= 70:
        test.loc[len(test)] = [name, matched_name, max_score]
    else:
        test.loc[len(test)] = [name, matched_name, max_score]

In [81]:
def remove_re_patterns(string):
    volume_pattern = r'(- \d+\s*(ml|ML|l|L))'  # Match digits followed by "ml,", "ML", "L," or "l"
    year_pattern = r'( \b\d{4}\b)'  # Match year/vintage
    new_string = re.sub(volume_pattern, '', string, flags=re.IGNORECASE) # case-insensitive
    new_string = re.sub(year_pattern, '', string)
    new_string = re.sub('[^a-zA-Z0-9 \n\.]', '', new_string)
    return new_string

In [92]:
test2 = pd.DataFrame(columns=['winelistasia_pdt_name', 'match_w_regex', 'max_score_w_regex'])

for name in product_names:
    edited_name = remove_re_patterns(name)
    substrings = edited_name.split(' ')

    possible_matches = []
    for substring in substrings:
        if substring != '':
            possible_matches += [key for key in bbw.keys() if substring in key]
    unique_possible_matches = set(possible_matches)
    
    max_score = 0
    matched_name = ''
    for wine in list(unique_possible_matches):
        score = fuzz.ratio(name.lower(), wine.lower())
        if score > max_score:
            matched_name = wine
            max_score = score
    if max_score >= 70:
        test2.loc[len(test2)] = [name, matched_name, max_score]
    else:
        test2.loc[len(test2)] = [name, matched_name, max_score]

## Compare Results
- whole thing runs faster with regex
- maybe fuzz.ratio takes a while?
- now check the actual matching -> does it give the same result? (it defo should la)

In [93]:
print(test.shape) # without regex
print(test2.shape) # with regex

(342, 3)
(342, 3)


In [95]:
merged = test.merge(test2, on='winelistasia_pdt_name')
print(merged.shape)

(342, 5)


In [96]:
merged

,winelistasia_pdt_name,match_wo_regex,max_score_wo_regex,match_w_regex,max_score_w_regex
0,Sartori - Soave Classico DOC 2022,Sartori Soave Classico DOC 2017,91,Sartori Soave Classico DOC 2017,91
1,Sartori - Pinot Grigio delle Venezie DOC 2022,Cesari Pinot Grigio Fiorile delle Venezie 2020...,74,Cesari Pinot Grigio Fiorile delle Venezie 2020...,74
2,"Sartori - Pinot Grigio ""Arcole"" DOC 2018",Freixenet Pinot Grigio Garda DOC 2021,68,Freixenet Pinot Grigio Garda DOC 2021,68
3,"Sartori - Soave Classico ""Sella"" DOC 2022",Sartori Soave Classico DOC 2017,81,Sartori Soave Classico DOC 2017,81
4,"Sartori - Bianco Veronese ""Marani"" IGT 2021","Sartori Bianco Veronese ""Marani"" IGT 2021",98,"Sartori Bianco Veronese ""Marani"" IGT 2021",98
...,...,...,...,...,...
337,"Eugenio Collavini - Merlot ""Di Casa"" DOC 2019","Eugenio Collavini Spumante ""Il Grigio"" NV",60,"Eugenio Collavini Spumante ""Il Grigio"" NV",60
338,Eugenio Collavini - Friulano DOC 2021,Unico Zelo Alluvium Fiano 2021,63,"Eugenio Collavini Spumante ""Il Grigio"" NV",62
339,Domaine La Biscarelle - Chateaunouf Du Pape AO...,Chateau de la Font du Loup Chateauneuf du Pape...,60,Domaines OTT Clos Mireille Rosé Coeur de Grain...,60
340,"Domaine La Biscarelle - Chateaunouf Du Pape ""L...",Famille Perrin Châteauneuf-du-Pape Les Sinards...,58,Famille Perrin Châteauneuf-du-Pape Les Sinards...,58
